In [4]:
import pandas as pd

csv_url = "https://health-infobase.canada.ca/src/data/covidLive/covid19.csv"
df = pd.read_csv(csv_url)


In [5]:
#parse government data so it is usable
df['dates'] = pd.to_datetime(df['date'], dayfirst = True)
df['YMD'] = df['dates'].dt.date
df = df.drop(['date', 'dates', 'prnameFR', 'pruid'], axis = 1)
#if provinces don't report active cases, assume they have 0, so the data will plot
df['numactive'].fillna(0.0, inplace = True)

#List of unique provinces, Canada, ordered alphabetically with Canada first
provnames = df.prname.unique()
provnames = list(provnames)
provnames.pop()
provnames.sort()
myorder = [2, 0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
provnames = [provnames[i] for i in myorder]
dropnames = ['Canada', 'Nunavut', 'Yukon', 'Northwest Territories']
select_provs = [x for x in provnames if x not in dropnames]

#function to make province-wise dataset
def make_dataset(pnames):
    
    by_prov = pd.DataFrame()

    # Iterate through all the provinces
    for i in pnames:

        # Subset to the province
        sub = df[df['prname'] == i]

        # Add to the overall dataframe
        by_prov = by_prov.append(sub)
    
    return by_prov

#must pivot datasets for linear plot data so prov names are columns
def format_dataset(df, value):
    df_pivot = pd.pivot_table(df,
    columns = 'prname',
    index = 'YMD',
    values = value)
    
    df_pivot = df_pivot.rename_axis(None)

    return df_pivot

In [6]:
dataset = format_dataset(make_dataset(select_provs), 'numtoday')
index_list = pd.date_range(start = '2021-02-26', end = '2021-03-26', closed = 'right')
month_period = dataset.loc[index_list, :]
print(month_period)
month_total = month_period.sum(axis = 0)
print(month_total)
#month_total.to_csv('data/29JAN_19FEB_totals.csv', encoding = 'utf8')

prname      Alberta  British Columbia  Manitoba  New Brunswick  \
2021-02-27    415.0             569.0      90.0            2.0   
2021-02-28    301.0             545.0      50.0            0.0   
2021-03-01    291.0             475.0      35.0            1.0   
2021-03-02    257.0             438.0      56.0            4.0   
2021-03-03    402.0             542.0      50.0            3.0   
2021-03-04    331.0             564.0      51.0            5.0   
2021-03-05    411.0             634.0      48.0            4.0   
2021-03-06    341.0             545.0      70.0            6.0   
2021-03-07    304.0             532.0      56.0            2.0   
2021-03-08    278.0             385.0      59.0            4.0   
2021-03-09    255.0             550.0      60.0            1.0   
2021-03-10    399.0             531.0      74.0            0.0   
2021-03-11    364.0             569.0      91.0            2.0   
2021-03-12    425.0             648.0      96.0            3.0   
2021-03-13